In [25]:
from pyomo.environ import *
from pyomo.contrib.piecewise import PiecewiseLinearFunction
from pyomo.core.base import TransformationFactory
from pyomo.opt import SolverStatus, TerminationCondition
#from pyomo.kernel import *
#from pyomo.core.kernel.piecewise_library.transforms_nd import piecewise_nd
#from pyomo.core.kernel.piecewise_library import util as pwutil  # 方便自动生成剖分
from dataclasses import dataclass, field
from typing import Callable, List, Dict, Tuple
import numpy as np
import math
import matplotlib.pyplot as plt
import bisect
import itertools as it
#from pyomo.environ import ConcreteModel, Var, Constraint, Expression

In [50]:
def evaluate_Q_at(model, first_stg_vars, first_stg_vals, solver):
    """
    Given y = y_val , minimize obj_expr and return v(y).
    This function temporarily increments the objective and clears it after completion, without changing the model structure.
    """
    # Clear any remaining As/pw/obj (to prevent it from being left over from the previous round)
    del_components(model)
    
    for u, v in zip(first_stg_vars, first_stg_vals):
        u.fix(value(v))
    model.obj = Objective(expr=model.obj_expr, sense=minimize)
    results = solver.solve(model, tee=False)

    status_ok = (results.solver.status == SolverStatus.ok)
    term_ok = (results.solver.termination_condition == TerminationCondition.optimal)
    if not (status_ok and term_ok):
        # check if solution okay
        raise RuntimeError(f"Scenario evaluate at y={first_stg_vals} not optimal: "
                           f"status={results.solver.status}, term={results.solver.termination_condition}")

    v_opt = value(model.obj_expr)
    # clear temporarily objective
    model.del_component('obj')
    for u in first_stg_vars:
        u.unfix()
    return v_opt

def del_components(model):
    for comp in ['obj', 'As', 'pw', 'pw_fun', 'pw_As', 'pw_link']:
        if hasattr(model, comp):
            model.del_component(comp)

def corners_from_bounds(firt_stg_vars):
    """给一组 Pyomo Var 生成所有 box 角点（每维取 lb/ub）"""
    bounds = []
    for y in firt_stg_vars:
        lb, ub = y.lb, y.ub
        if lb is None or ub is None:
            raise ValueError(f"{y.name} 缺少上下界，无法生成角点")
        bounds.append((float(lb), float(ub)))
    # 每维挑 lb/ub 的笛卡尔积
    return list(it.product(*[(lb, ub) for (lb, ub) in bounds]))


def add_nd_piecewise(
    model,
    firt_stg_vars,                 # [x1, x2, ..., xN]  模型里的 first stage Var
    points,                 # [(c11,...,c1N), (c21,...,c2N), ...]  所有节点（同维度 N）
    values,                 # 与 points 对齐的一维 list/array，或 dict{point_tuple: value}
    name="pw",
    relation="==",          # '==', '>='(下界/内逼近), '<='(上界/外逼近)
    round_ndigits=12,       # 为避免浮点比较问题，对坐标做轻微 round
):
    """
    返回 (z, pw)。z 是 Var（或你可将 make_z_var=False 改成返回 Expression）。
    """

    # 维度检查
    if len(points) == 0:
        raise ValueError("points 不能为空")
    N = len(firt_stg_vars)
    for pt in points:
        if len(pt) != N:
            raise ValueError(f"points 中出现与 x_vars 维度不一致的点: {pt}")
        
    del_components(model)

    # 统一坐标的浮点表示，避免查表时精度问题
    def keyize(coords):
        return tuple(round(float(c), round_ndigits) for c in coords)

    norm_points = [keyize(pt) for pt in points]

    # 把 values 统一成 dict 表
    if isinstance(values, dict):
        table = {keyize(k): float(v) for k, v in values.items()}
        # 确保每个 point 都有值
        miss = [pt for pt in norm_points if pt not in table]
        if miss:
            raise KeyError(f"values 缺少这些点的取值: {miss[:5]}{' ...' if len(miss)>5 else ''}")
    else:
        # 视作与 points 对齐的一维序列
        if len(values) != len(points):
            raise ValueError("values 长度应与 points 数量一致（或传 dict）")
        table = {pt: float(v) for pt, v in zip(norm_points, values)}

    # 查表函数（仅在节点上被调用）
    def _f_from_table(*coords):
        return table[keyize(coords)]

    # 创建并挂到模型
    pw = PiecewiseLinearFunction(points=norm_points, function=_f_from_table, name=f"{name}_fun")
    model.add_component(pw.name, pw)

    # 组成表达式
    pw_expr = pw(*firt_stg_vars)

    As = Var(name=f"{name}_As")
    model.add_component(As.name, As)
    if relation == "==":
        link = Constraint(expr=As == pw_expr)
    elif relation == ">=":
        link = Constraint(expr=As >= pw_expr)
    elif relation == "<=":
        link = Constraint(expr=As <= pw_expr)
    else:
        raise ValueError("relation 只能是 '==', '>=', '<='")
    model.add_component(f"{name}_link", link)

    TransformationFactory('contrib.piecewise.convex_combination').apply_to(model)
        
    return As, pw


def clone_and_get_vars(m_old, first_stage_vars):
    """
    克隆模型，并按名字列表返回新模型中的变量组件
    var_names: ['y', 'x', ...]  (只能是容器名，不是元素)
    """
    m_new = m_old.clone()
    first_stage_vars_new = []
    for v in first_stage_vars:
        v_new = m_new.find_component(v.name)
        if v_new is None:
            raise KeyError(f"在新模型里找不到变量 '{v.name}'")
        first_stage_vars_new.append(v_new)
    return m_new, first_stage_vars_new

# delete repeated nodes
def unique_points(points, atol=1e-9):
    out = []
    for p in points:
        if not any(all(abs(a-b) <= atol for a,b in zip(p, q)) for q in out):
            out.append(p)
    return out


def nc_underest(model_list, first_stg_vars_list, m_tmpl_list, add_node_num, picture_shown=False, v_list=False, tolerance=1e-8):
    """
    Parameters:
        #bounds (list): contains 2 float which is lower and upper bound of variable
        model_list (list): model with submodels corresponds to each scenario
        first_stg_var (list): 
        m_tmpl_list (list): [template model, template model first stg variables list]
        add_node_num (float): number of sample nodes
        tolerance (float): decide when to stop

    Returns: delta (float): delta
             errors (float): hausdorff error
             y_nodes (list): y node (to make plot)
             as_nodes_list[0] (list): As node value (to make plot)
             ms_list[0] (float): ms for first scenario (to make plot)
    """
    N = len(model_list)
    as_nodes_list = [[] for _ in range(N)]
    ms_list = [None] * N
    new_nodes_list = [None] * N # Storing potential new nodes
    As_min_list = []
    under_tol = 1e-8
    
    add_node_history = []

    # set up solver
    solver = SolverFactory('gurobi')
    solver.options.update({
        'MIPGap': 1e-8,         
        'MIPGapAbs': 0.0,       
        'FeasibilityTol': 1e-9,  
        'IntFeasTol':     1e-9,  
        'OptimalityTol': 1e-9,
        'NumericFocus': 2,      
        'ScaleFlag':    1,       
        'Presolve': 2,          
        'Method':  -1,          
        'Crossover': -1,       
        'NonConvex': 2, 
    })
    
    ######### if we want to plot figures #########
    if picture_shown:
        y_vals = np.linspace(new_lb, new_ub, 100)
        Qs_vals_list = [None] * N
        for i in range(N):
            Qs_vals_list[i] = [v_list[i](y) for y in y_vals]
        Qs_arr = np.array(Qs_vals_list, dtype=float, ndmin=2)  
        Qs_vals_sum = Qs_arr.sum(axis=0)
    #############################################
    
    # start from corner nodes
    first_stg_nodes = corners_from_bounds(first_stg_vars_list[0])
    for i in range(N):
        as_nodes_list[i].extend(
            evaluate_Q_at(model_list[i], first_stg_vars_list[i], node, solver) for node in first_stg_nodes
        )
    '''
    for node in first_stg_nodes:
        for i in range(N):
            as_nodes_list[i].append(evaluate_Q_at(model_list[i], node, solver))
    '''

    
    print('The goal is to get ',add_node_num,' nodes')
    for k in range(3,add_node_num+1):
        print('##################################################')
        print('##################################################')
        print('Start adding node ',k)
        for i in range(N):
            print(' ')
            print('Solving scenario ',i)
            # define piecewise function for each scenario
            del_components(model_list[i])

            As, _ = add_nd_piecewise(
            model_list[i], first_stg_vars_list[i], first_stg_nodes, as_nodes_list[i],
            name="pw", relation="=="
            )
            model_list[i].obj = Objective(expr=model_list[i].obj_expr - As, sense=minimize)
            #results = SolverFactory("gurobi").solve(model_list[i], tee=False)
            results = solver.solve(model_list[i], tee=False)
            if (results.solver.status != SolverStatus.ok) or \
               (results.solver.termination_condition != TerminationCondition.optimal):
                print("⚠ There may be problems with the solution")
                
            ms_list[i] = value(model_list[i].obj)
            # insert new nodes
            new_nodes_list[i] = tuple(value(v) for v in first_stg_vars_list[i])

        #####################################################################################################
        ##################################### picture show ##################################################
            if picture_shown:
                print(' ')
                print('The plot for scenario ',i)
                print('The potential y_star is ',value(model_list[i].y))
                print('ms is ',value(model_list[i].obj))
                # set up plot parameters
                y_nodes_arr = np.array(y_nodes)
                as_nodes_arr = np.array(as_nodes_list[i])
                y_star_i = value(model_list[i].y)
                ms_i = value(model_list[i].obj)
                ## plot the figure
                plt.figure(figsize=(8, 5))
                plt.plot(y_vals, Qs_vals_list[i], label=fr'$Qs_{i}$', color='red')
                plt.plot(y_nodes_arr, as_nodes_arr+ms_i,label=fr'$As_{i} underest$',color='red', marker='o',linestyle='--',alpha=0.5)
                plt.plot(y_nodes_arr, as_nodes_arr,label=fr'$As_{i}$',color='blue', marker='o',linestyle='--',alpha=0.5)
                plt.axvline(x=y_star_i, color='purple', linestyle='--')
                plt.xlim(new_lb, new_ub)
                plt.xlabel('y')
                plt.ylabel('value')
                plt.title(fr"Plot for scenario {i} for {k} nodes")
                plt.legend()
                plt.grid(True)
                plt.tight_layout()
                plt.show()
        ###################################### picture show end #############################################
        #####################################################################################################

        # define and solve the sum model
        arr = np.array(as_nodes_list, dtype=float, ndmin=2)  
        assum_nodes = arr.sum(axis=0) 

        # build As_sum model and solve for possible node pf max error
        model_sum, model_sum_first_stg_vars = clone_and_get_vars(m_tmpl_list[0], m_tmpl_list[1])
        del_components(model_sum)
        As, _ = add_nd_piecewise(
        model_sum, model_sum_first_stg_vars, first_stg_nodes, assum_nodes,
        name="pw", relation="=="
        )
        model_sum.obj = Objective(expr= As, sense=minimize)
        #results = SolverFactory("gurobi").solve(model_sum, tee=False)
        results = solver.solve(model_sum, tee=False)
        if (results.solver.status == SolverStatus.ok) and (results.solver.termination_condition == TerminationCondition.optimal):
            pass
        else:
            print("Sum model doesn't get solved normally")

        # get the output
        As_min = results.problem.lower_bound
        node_star = tuple(value(v) for v in model_sum_first_stg_vars) 
        '''
        if y_star is None:
            y_star = 0.5*(new_lb + new_ub)
        '''
        errors_node_star = 0
        for i in range(N):
            errors_node_star += evaluate_Q_at(model_list[i], first_stg_vars_list[i], node_star, solver)
        errors_node_star = abs(As_min - errors_node_star)

        #####################################################################################################
        ##################################### picture show ##################################################
        if picture_shown:
            print(' ')
            print('The plot for As_sum')
            print('The potential y_star is ',y_star)
            print('error is ',errors_y_star)
            # set up plot parameters
            y_nodes_arr = np.array(y_nodes)
            assum_nodes_arr = np.array(assum_nodes)
            ms_sum = sum(ms_list)
            ## plot the figure
            plt.figure(figsize=(8, 5))
            plt.plot(y_vals, Qs_vals_sum, label=fr'$Qs_sum$', color='red')
            plt.plot(y_nodes_arr, assum_nodes_arr+ms_sum,label=fr'$As_sum underest$',color='red', marker='o',linestyle='--',alpha=0.5)
            plt.plot(y_nodes_arr, assum_nodes_arr,label=fr'$As_sum$',color='blue', marker='o',linestyle='--',alpha=0.5)
            plt.axvline(x=y_star, color='purple', linestyle='--')
            plt.xlim(new_lb, new_ub)
            plt.xlabel('y')
            plt.ylabel('value')
            plt.title(fr"Plot for As_sum for {k} nodes")
            plt.legend()
            plt.grid(True)
            plt.tight_layout()
            plt.show()
        ###################################### picture show end #############################################
        #####################################################################################################

        sum_ms = sum(ms_i for ms_i in ms_list)
        
        print('Sum *****************************************')
        print('error at y_star is ',errors_node_star)
        print('y_star is ',node_star)
        print('ms_list and sum_ms is ',ms_list,sum_ms)
        if errors_node_star > abs(sum_ms):
            new_node = node_star
            print('new node choosen from error')
        else:
            min_index = np.argmin(ms_list)
            new_node = new_nodes_list[min_index]
            print('new node choosen from ms')
        As_min_list.append(As_min+sum_ms)
        add_node_history.append(new_node)
        print('new node is',new_node)
        print('Current As_min is',As_min_list[-1])
        print('*****************************************')
        print('')
        #######################################################              

        first_stg_nodes.append(new_node)
        for i in range(N):
            as_nodes_list[i].append(evaluate_Q_at(model_list[i], first_stg_vars_list[i], new_node, solver))

    # define and solve the sum model
    arr = np.array(as_nodes_list, dtype=float, ndmin=2)  
    assum_nodes = arr.sum(axis=0) 

    # build As_sum model and solve for possible node pf max error
    model_sum, model_sum_first_stg_vars = clone_and_get_vars(m_tmpl_list[0], m_tmpl_list[1])
    del_components(model_sum)
    As, _ = add_nd_piecewise(
    model_sum, model_sum_first_stg_vars, first_stg_nodes, assum_nodes,
    name="pw", relation="=="
    )
    model_sum.obj = Objective(expr= As, sense=minimize)
    #results = SolverFactory("gurobi").solve(model_sum, tee=False)
    results = solver.solve(model_sum, tee=False)
    if (results.solver.status == SolverStatus.ok) and (results.solver.termination_condition == TerminationCondition.optimal):
        pass
    else:
        print("Sum model doesn't get solved normally")

    # get the output
    output_lb = results.problem.lower_bound + sum(ms_list)
    
    return output_lb, first_stg_nodes, [range(3,add_node_num+1), As_min_list, add_node_history]

In [51]:
# setup models and parameters

# bounds
ylb, yub = -0.2, 0.2
m1 = ConcreteModel()
m1.y1 = Var(bounds=(ylb, yub))
m1.y2 = Var(bounds=(ylb, yub))
m1.x1  = Var(bounds=(0, None))   # = sqrt(x11)
m1.x11 = Var(bounds=(0, None))   # = sqrt(y1^2 + y2^2)
m1.c1 = Constraint(expr = m1.x11**2 == m1.y1**2 + m1.y2**2)
m1.c2 = Constraint(expr = m1.x1**2  == m1.x11)
m1.obj_expr = Expression(expr = -m1.x1)

m2 = m1.clone()
m_tmpl = ConcreteModel()
m_tmpl.y1 = Var(bounds=(-0.2, 0.2))
m_tmpl.y2 = Var(bounds=(-0.2, 0.2))
m_tmpl_list = [m_tmpl, [m_tmpl.y1, m_tmpl.y2]]

model_list = [m1,m2]
first_stg_vars_list = [[m1.y1, m1.y2],[m2.y1, m2.y2]]



In [52]:
nc_underest(model_list, first_stg_vars_list, m_tmpl_list, add_node_num=5,
                picture_shown=False, v_list=False, tolerance=1e-8
)

The goal is to get  5  nodes
##################################################
##################################################
Start adding node  3
 
Solving scenario  0
 
Solving scenario  1
Sum *****************************************
error at y_star is  0.0
y_star is  (-0.2, -0.2)
ms_list and sum_ms is  [0.0, 0.0] 0.0
new node choosen from ms
new node is (-0.2, -0.2)
Current As_min is -1.0636591793889978
*****************************************

##################################################
##################################################
Start adding node  4
 
Solving scenario  0


KeyError: 4

In [4]:
from pyomo.kernel import *

# build function for each scenario
def v_1(y):  
    return 0.5*y**2

v_list = [v_1]

# lower bound and upper bound
ylb = -1
yub = 1
# build model for each scenario 
m1 = ConcreteModel()
m1.y = Var(bounds=(ylb, yub))
m1.x1 = Var()
m1.c1 = Constraint(expr=m1.x1 == m1.y**2)
m1.obj_expr = Expression(expr=0.5*m1.x1)

model_list = [m1]
# make delta list
result = nc_underest([ylb, yub],model_list,20,True, v_list)

NameError: name 'ConcreteModel' is not defined

In [5]:
import itertools as it

def add_nd_piecewise(
    model: ConcreteModel,
    z: Var,                 # 目标：f(x) 的值变量
    xs: list[Var],          # 自变量列表 [x1, x2, ..., xN]
    grids: list[list[float]],   # 各维分段点 [[...], [...], ...]
    f_rule=None,                # callable(*coords)->float  或
    f_table: dict=None,         # {(x1,...,xN): fval, ...}
    constr_type: str = "EQ",    # 'EQ' 常用
    repn: str = "SOS2"          # 'SOS2' | 'BIGM' | 'CC'
):
    """
    在 model 上添加多维 Piecewise: z ≈ f(xs) on grids。
    你必须二选一提供 f_rule 或 f_table。
    """
    assert (f_rule is not None) ^ (f_table is not None), "Provide exactly one of f_rule or f_table"
    assert len(xs) == len(grids) and len(xs) >= 1, "xs & grids must align and be non-empty"

    # Pyomo 的多维 Piecewise 直接传 [x1,...,xN] 和 pw_pts=[pts1,...,ptsN]
    if f_table is None:
        # 直接给函数
        model.pw = Piecewise(
            z, xs,
            pw_pts=grids,
            f_rule=lambda *coords: float(f_rule(*coords)),
            pw_constr_type=constr_type,
            pw_repn=repn
        )
    else:
        # 表格：键用坐标元组。若用户给的是索引键，这里转坐标键
        # 允许用户用真实坐标键 {(x1,...,xN): val} ——最简单
        # 若用户提供的是“索引键”，也可改写为从索引转坐标。
        # 这里假定 f_table 的键就是坐标元组：
        def _frule(*coords):
            return float(f_table[tuple(coords)])
        model.pw = Piecewise(
            z, xs,
            pw_pts=grids,
            f_rule=_frule,
            pw_constr_type=constr_type,
            pw_repn=repn
        )
    return model.pw

In [7]:
# 例：f(x,y,w) = x^2 + y^2 + w^2，在[0,0.5,1]^3 网格做分段
m = ConcreteModel()
m.x = Var(bounds=(0,1))
m.y = Var(bounds=(0,1))
m.w = Var(bounds=(0,1))
m.f = Var()

grids = [[0,0.5,1], [0,0.5,1], [0,0.5,1]]
f = lambda x,y,w: x**2 + y**2 + w**2

add_nd_piecewise(m, m.f, [m.x, m.y, m.w], grids, f_rule=f, repn="SOS2")

# 举个优化：最小化 f，附加一个线性约束
m.obj = Objective(expr=m.f, sense=pyo.minimize)
m.c1  = Constraint(expr = m.x + m.y + m.w >= 1.2)

# 用支持 SOS2 的 MIP 求解器（如 Gurobi/CPLEX/HiGHS(不支持SOS2，请改 BIGM)）
solver = SolverFactory("gurobi")      # 或 cplex
res = solver.solve(m, tee=False)
print(value(m.x), value(m.y), value(m.w), value(m.f))


TypeError: Piecewise component has invalid argument type for domain variable, [<pyomo.core.base.var.ScalarVar object at 0x7f7e411f8a50>, <pyomo.core.base.var.ScalarVar object at 0x7f7e411f8b30>, <pyomo.core.base.var.ScalarVar object at 0x7f7e411f8ac0>]

In [8]:
import numpy as np
from scipy.spatial import Delaunay
import pyomo.environ as pyo
import pyomo.kernel as pmo
from pyomo.core.kernel.piecewise_library.transforms_nd import piecewise_nd

def add_nd_piecewise_kernel(model, z_var, x_vars, grids, f_rule, bound='eq'):
    """
    用 kernel.piecewise_nd 在任意维度上构建分段线性逼近：
        z_var  =  f_rule(x_vars)  在 grids 的笛卡尔网格上线性插值
    参数:
        model  : pyo.ConcreteModel
        z_var  : pyo.Var （输出变量）
        x_vars : [pyo.Var, ...] 长度 = 维度D
        grids  : [[pts_dim1], [pts_dim2], ...]
        f_rule : 可调用，签名 f(*coords) -> float
        bound  : 'eq' | 'lb' | 'ub'
    """
    D = len(x_vars)
    # 1) 生成笛卡尔网格上的离散点
    mesh = np.array(np.meshgrid(*grids, indexing='ij'))
    points = mesh.reshape(D, -1).T  # 形状 (npoints, D)

    # 2) 对网格做 Delaunay 三角剖分
    tri = Delaunay(points)

    # 3) 计算每个网格点的函数值
    values = np.array([float(f_rule(*pt)) for pt in points])

    # 4) 构建 kernel piecewise_nd 约束
    #    注意：需要 kernel 表达式把 x_vars / z_var “挂接”进去
    blk = pmo.block()
    blk_in = [pmo.expression(x) for x in x_vars]
    blk_out = pmo.expression(z_var)

    # 创建并附着到模型（作为子块）
    name = f"pw_nd_{id(blk)}"
    setattr(model, name, blk)

    # 在该块内添加多维 PW 关系（CC 表示）
    pw_block = piecewise_nd(tri, values, input=blk_in, output=blk_out, bound=bound, repn='cc')
    blk.add(pw_block)

    return pw_block  # 可用于调试/检查

# ======= 示例：3维 f(x,y,w) = x^2 + y^2 + w^2 =======
m = pyo.ConcreteModel()
m.x = pyo.Var(bounds=(0,1))
m.y = pyo.Var(bounds=(0,1))
m.w = pyo.Var(bounds=(0,1))
m.f = pyo.Var()

grids = [[0, 0.5, 1], [0, 0.5, 1], [0, 0.5, 1]]
f = lambda x, y, w: x**2 + y**2 + w**2

# 关键：用 kernel 的 piecewise_nd，而不是 environ 的 Piecewise
add_nd_piecewise_kernel(m, m.f, [m.x, m.y, m.w], grids, f_rule=f, bound='eq')

# 目标+约束（示例）
m.obj = pyo.Objective(expr=m.f, sense=pyo.minimize)
m.c1 = pyo.Constraint(expr=m.x + m.y + m.w >= 1.2)

solver = pyo.SolverFactory("gurobi")  # 或 cplex/scip；该表示是“凸组合(CC)”，不依赖 SOS2
solver.solve(m, tee=False)

print("x,y,w,f* =", pyo.value(m.x), pyo.value(m.y), pyo.value(m.w), pyo.value(m.f))


AttributeError: 'block' object has no attribute 'add'

In [9]:
import numpy as np
from scipy.spatial import Delaunay
import pyomo.environ as pyo
import pyomo.kernel as pmo
from pyomo.core.kernel.piecewise_library.transforms_nd import piecewise_nd

def add_nd_piecewise_kernel(model, z_var, x_vars, grids, f_rule, bound='eq'):
    D = len(x_vars)

    # 1) 生成笛卡尔网格点
    mesh = np.array(np.meshgrid(*grids, indexing='ij'))
    points = mesh.reshape(D, -1).T

    # 2) Delaunay 三角剖分
    tri = Delaunay(points)

    # 3) 点上的函数值
    values = np.array([float(f_rule(*pt)) for pt in points])

    # 4) 建 kernel block，并把 environ 变量包装成 expression
    blk = pmo.block()
    blk_in = []
    for v in x_vars:
        e = pmo.expression()
        e.expr = v
        blk_in.append(e)

    blk_out = pmo.expression()
    blk_out.expr = z_var

    # 5) 生成 N 维分段线性（凸组合/CC表示）
    pw_block = piecewise_nd(tri, values, input=blk_in, output=blk_out, bound=bound, repn='cc')

    # 6) 正确挂接（不要 .add）
    blk.pw = pw_block

    # 7) 把整个 kernel 子块挂到模型上
    setattr(model, f"pw_nd_{id(blk)}", blk)
    return pw_block


In [10]:
m = pyo.ConcreteModel()
m.x = pyo.Var(bounds=(0,1))
m.y = pyo.Var(bounds=(0,1))
m.w = pyo.Var(bounds=(0,1))
m.f = pyo.Var()

grids = [[0,0.5,1],[0,0.5,1],[0,0.5,1]]
f = lambda x,y,w: x**2 + y**2 + w**2

add_nd_piecewise_kernel(m, m.f, [m.x, m.y, m.w], grids, f_rule=f, bound='eq')

m.obj = pyo.Objective(expr=m.f, sense=pyo.minimize)
pyo.SolverFactory("gurobi").solve(m, tee=False)
print(pyo.value(m.x), pyo.value(m.y), pyo.value(m.w), pyo.value(m.f))


model.name="unknown";
    - termination condition: infeasibleOrUnbounded
    - message from solver: Problem proven to be infeasible or unbounded.
ERROR: evaluating object as numeric value: x
        (object: <class 'pyomo.core.base.var.ScalarVar'>)
    No value for uninitialized NumericValue object x


ValueError: No value for uninitialized NumericValue object x

In [11]:
import numpy as np
from scipy.spatial import Delaunay
import pyomo.environ as pyo
import pyomo.kernel as pmo
from pyomo.core.kernel.piecewise_library.transforms_nd import piecewise_nd

def add_nd_piecewise_kernel(model, z_var, x_vars, grids, f_rule, bound='eq'):
    D = len(x_vars)
    mesh = np.array(np.meshgrid(*grids, indexing='ij'))
    points = mesh.reshape(D, -1).T
    tri = Delaunay(points)
    values = np.array([float(f_rule(*pt)) for pt in points])

    blk = pmo.block()
    blk_in = []
    for v in x_vars:
        e = pmo.expression()
        e.expr = v
        blk_in.append(e)
    blk_out = pmo.expression(); blk_out.expr = z_var

    blk.pw = piecewise_nd(tri, values, input=blk_in, output=blk_out, bound=bound, repn='cc')
    setattr(model, f"pw_nd_{id(blk)}", blk)
    return blk.pw

# ---- model ----
m = pyo.ConcreteModel()
m.x = pyo.Var(bounds=(0,1))
m.y = pyo.Var(bounds=(0,1))
m.w = pyo.Var(bounds=(0,1))
m.f = pyo.Var()

grids = [[0,0.5,1],[0,0.5,1],[0,0.5,1]]
f = lambda x,y,w: x**2 + y**2 + w**2

add_nd_piecewise_kernel(m, m.f, [m.x, m.y, m.w], grids, f_rule=f, bound='eq')

# 加一条简单约束，防止最优点退到(0,0,0)
m.c1 = pyo.Constraint(expr = m.x + m.y + m.w >= 1.2)
m.obj = pyo.Objective(expr=m.f, sense=pyo.minimize)

# **关键：展开 kernel 到 environ**
pyo.TransformationFactory('core.expand_kernel').apply_to(m)

# 求解
solver = pyo.SolverFactory("gurobi")  # 或 cplex/scip/highs
res = solver.solve(m, tee=False)

print("status:", res.solver.termination_condition)
print(pyo.value(m.x), pyo.value(m.y), pyo.value(m.w), pyo.value(m.f))


AttributeError: 'NoneType' object has no attribute 'apply_to'

In [3]:
import numpy as np
import pyomo.environ as pyo          # 仅用于调用求解器
import pyomo.kernel as pmo
from pyomo.core.kernel.piecewise_library.transforms_nd import piecewise_nd
from pyomo.core.kernel.piecewise_library import util as pwutil  # 方便自动生成剖分

# ---------- Kernel 模型 ----------
m = pmo.block()

# 变量（kernel 写法）
m.x = pmo.variable(lb=0, ub=1)
m.y = pmo.variable(lb=0, ub=1)
m.w = pmo.variable(lb=0, ub=1)
m.z = pmo.variable()

# 目标函数（我们用分段线性近似来表达它）
def f(x, y, w):
    return x**2 + y**2 + w**2

# 生成 N 维三角剖分（在各维的 [lb, mid, ub] 上采样）
var_list = [m.x, m.y, m.w]
tri = pwutil.generate_delaunay(var_list, num=3)   # 每维3个点：lb/mid/ub
values = f(*tri.points.T)                         # 各格点上的函数值

# 多维分段线性关系（凸组合/cc 表示，通用 MILP，任何 MILP 求解器都可）
m.pw = piecewise_nd(tri, values, input=var_list, output=m.z, bound='eq', repn='cc')

# 一条简单约束，防止最优点退到(0,0,0)
m.c1 = pmo.constraint(expr=m.x + m.y + m.w >= 1.2)

# 目标
m.obj = pmo.objective(expr=m.z, sense=pmo.minimize)

# ---------- 求解 ----------
res = pyo.SolverFactory("gurobi").solve(m, tee=False)   # 也可用 glpk/cbc/scip/cplex（MILP）
print("term:", res.solver.termination_condition)
print("x y w z* =", m.x.value, m.y.value, m.w.value, m.z.value)


term: optimal
x y w z* = 0.35 0.35 0.5 0.5999999999999999


In [4]:
tri.points.T

array([[0. , 0. , 0. , 0.5, 0.5, 0.5, 1. , 1. , 1. , 0. , 0. , 0. , 0.5,
        0.5, 0.5, 1. , 1. , 1. , 0. , 0. , 0. , 0.5, 0.5, 0.5, 1. , 1. ,
        1. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0.5, 0.5, 0.5,
        0.5, 0.5, 0.5, 0.5, 0.5, 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. ,
        1. ],
       [0. , 0.5, 1. , 0. , 0.5, 1. , 0. , 0.5, 1. , 0. , 0.5, 1. , 0. ,
        0.5, 1. , 0. , 0.5, 1. , 0. , 0.5, 1. , 0. , 0.5, 1. , 0. , 0.5,
        1. ]])

In [5]:
# build function for each scenario
def v_1(y):  
    return - np.sqrt(np.abs(y))

def v_2(y): 
    return np.sqrt(np.abs(y))

v_list = [v_1, v_2]

# lower bound and upper bound
ylb = -0.2
yub = 0.2
bounds = [ylb, yub]
# build model for each scenario 
m1 = ConcreteModel()
m1.y1 = Var(bounds=(ylb, yub))
m1.y2 = Var(bounds=(ylb, yub))
m1.x1 = Var(bounds=(0, 1))
m1.x11 = Var(bounds=(0, None))
m1.c1 = Constraint(expr=m1.x11**2 == m1.y1**2)
m1.c2 = Constraint(expr=m1.x11 == (m1.x1)**2)
m1.c3 = Constraint(expr=m1.x11**2 == m1.y2**2)
m1.c4 = Constraint(expr=m1.x11 == (m1.x1)**2)
m1.obj_expr = Expression(expr=-m1.x1)

m2 = ConcreteModel()
m2.y1 = Var(bounds=(ylb, yub))
m2.y2 = Var(bounds=(ylb, yub))
m2.x2 = Var(bounds=(0, 1))
m2.c1 = Constraint(expr=m2.x2**2 >= m2.y1)
m2.c2 = Constraint(expr=m2.x2**2 >= -m2.y1)
m2.c3 = Constraint(expr=m2.x2**2 >= m2.y2)
m2.c4 = Constraint(expr=m2.x2**2 >= -m2.y2)
m2.obj_expr = Expression(expr=m2.x2)

model_list = [m1, m2]
first_stg_var_list = [m.x, m.y, m.w]

In [ ]:
# build function for each scenario
def v_1(y):  
    return - np.sqrt(np.abs(y))

def v_2(y): 
    return np.sqrt(np.abs(y))

v_list = [v_1, v_2]

# lower bound and upper bound
ylb = -0.2
yub = 0.2
bounds = [ylb, yub]
# build model for each scenario 
m1 = ConcreteModel()
m1.y1 = Var(bounds=(ylb, yub))
m1.y2 = Var(bounds=(ylb, yub))
m1.x1 = Var(bounds=(0, 1))
m1.x11 = Var(bounds=(0, None))
m1.c1 = Constraint(expr=m1.x11**2 == m1.y1**2)
m1.c2 = Constraint(expr=m1.x11 == (m1.x1)**2)
m1.c3 = Constraint(expr=m1.x11**2 == m1.y2**2)
m1.c4 = Constraint(expr=m1.x11 == (m1.x1)**2)
m1.obj_expr = Expression(expr=-m1.x1)

m2 = ConcreteModel()
m2.y1 = Var(bounds=(ylb, yub))
m2.y2 = Var(bounds=(ylb, yub))
m2.x2 = Var(bounds=(0, 1))
m2.c1 = Constraint(expr=m2.x2**2 >= m2.y1)
m2.c2 = Constraint(expr=m2.x2**2 >= -m2.y1)
m2.c3 = Constraint(expr=m2.x2**2 >= m2.y2)
m2.c4 = Constraint(expr=m2.x2**2 >= -m2.y2)
m2.obj_expr = Expression(expr=m2.x2)

model_list = [m1, m2]
first_stg_var_list = [m.x, m.y, m.w]

In [ ]:
# ---------- Kernel 模型 ----------
# lower bound and upper bound
ylb = -0.2
yub = 0.2
m1 = pmo.block()

# 变量（kernel 写法）
m1.y1  = pmo.variable(lb=ylb, ub=yub)
m1.y2  = pmo.variable(lb=ylb, ub=yub)
m1.x1  = pmo.variable(lb=0, ub=1)
m1.x11  = pmo.variable(lb=0, ub=None)
m1.c1 = pmo.constraint(expr=m1.x11**2 == m1.y1**2)
m1.c2 = pmo.constraint(expr=m1.x11 == (m1.x1)**2)
m1.c3 = pmo.constraint(expr=m1.x11**2 == m1.y2**2)
m1.c4 = pmo.constraint(expr=m1.x11 == (m1.x1)**2)

# 目标函数（我们用分段线性近似来表达它）
def f(y1,y2):
    return -np.sqrt(np.sqrt(y1**2 + y2**2))

# 生成 N 维三角剖分（在各维的 [lb, mid, ub] 上采样）
var_list = [m1.y1, m1.y2]
tri = pwutil.generate_delaunay(var_list, num=3)   # 每维3个点：lb/mid/ub
values = f(*tri.points.T)                         # 各格点上的函数值

# 多维分段线性关系（凸组合/cc 表示，通用 MILP，任何 MILP 求解器都可）
m1.pw = piecewise_nd(tri, values, input=var_list, output=m.z, bound='eq', repn='cc')

# 目标
m1.obj = pmo.objective(expr=m1.x1, sense=pmo.minimize)

# ---------- 求解 ----------
res = pyo.SolverFactory("gurobi").solve(m1, tee=False)   # 也可用 glpk/cbc/scip/cplex（MILP）
print("term:", res.solver.termination_condition)
print("x y w z* =", m1.y1.value, m1.y2.value, pmo.value(m1.obj))


term: optimal
x y w z* = 0.0 0.0 0.0


In [13]:
import pyomo.environ as pyo
import numpy as np
# 如果你要自动三角剖分二维网格，请确保安装了 scipy（pip install scipy）
from pyomo.contrib.piecewise import PiecewiseLinearFunction
from pyomo.core.base.transform import TransformationFactory

# ---------- 参数 ----------
ylb, yub = -0.2, 0.2

# ---------- 模型（pyo 版） ----------
m = pyo.ConcreteModel()
m.y1 = pyo.Var(bounds=(ylb, yub))
m.y2 = pyo.Var(bounds=(ylb, yub))
m.x1 = pyo.Var(bounds=(0, 1))
m.x11 = pyo.Var(bounds=(0, None))

# 原约束：|y1| = x11, |y2| = x11, 且 x11 = x1^2
# 注意：这些是非凸二次等式 -> 模型会是非凸 MIQCP
m.c1 = pyo.Constraint(expr=m.x11**2 == m.y1**2)
m.c2 = pyo.Constraint(expr=m.x11 == m.x1**2)
m.c3 = pyo.Constraint(expr=m.x11**2 == m.y2**2)
m.c4 = pyo.Constraint(expr=m.x11 == m.x1**2)  # 与 c2 相同，可删一条，这里保留与原代码一致

# 目标函数（用二维 piecewise 近似 z = f(y1,y2)）
def f(y1, y2):
    return -pyo.sqrt(pyo.sqrt(y1**2 + y2**2))  # 仅用于取值，不会进入求解器的非线性系统

# 采样点：各维 [lb, mid, ub]
grid = np.linspace(ylb, yub, 3)
points = [(a, b) for a in grid for b in grid]

# 定义分段线性函数组件（由 points 自动做 Delaunay 三角剖分并在每个单形上线性化）
m.pw = PiecewiseLinearFunction(function=lambda a, b: float(-np.sqrt(np.sqrt(a*a + b*b))),
                               points=points)

# 直接在表达式里调用 pw，得到 PiecewiseLinearExpression
z_expr = m.pw(m.y1, m.y2)

# 目标：最小化 z（更能体现分段近似的作用；如需保持你原来的“最小化 x1”，把这里换成 m.x1 即可）
m.obj = pyo.Objective(expr=z_expr, sense=pyo.minimize)

# 将包含 PiecewiseLinearFunction 的模型转换为 MIP（凸组合公式）
TransformationFactory('contrib.piecewise.convex_combination').apply_to(m)  # :contentReference[oaicite:1]{index=1}

# ---------- 求解 ----------
# 若含有二进制变量（由分段带来）+ 非凸二次约束，上 Gurobi 请设置 NonConvex=2
opt = pyo.SolverFactory("gurobi")
opt.options["NonConvex"] = 2
res = opt.solve(m, tee=False)

# ---------- 输出 ----------
print("status:", res.solver.termination_condition)
print("x1 =", pyo.value(m.x1), "x11 =", pyo.value(m.x11))
print("y1 =", pyo.value(m.y1), "y2 =", pyo.value(m.y2))
print("z* =", pyo.value(z_expr))


ModuleNotFoundError: No module named 'pyomo.core.base.transform'

In [14]:
import pyomo.environ as pyo
from pyomo.contrib.piecewise import PiecewiseLinearFunction
from pyomo.core.base import TransformationFactory

# ----- 模型 -----
m = pyo.ConcreteModel()
m.x = pyo.Var(bounds=(0, 1))
m.y = pyo.Var(bounds=(0, 1))
m.w = pyo.Var(bounds=(0, 1))
m.f = pyo.Var()  # z = piecewise(x,y,w)

# 采样网格（各维 0, 0.5, 1）
grid = [0.0, 0.5, 1.0]
points = [(a, b, c) for a in grid for b in grid for c in grid]

# 定义多维分段线性函数（自动对 points 做单形剖分）
m.pw = PiecewiseLinearFunction(
    points=points,
    function=lambda a, b, c: float(a*a + b*b + c*c)
)

# 将分段线性表达式绑定到变量 m.f
m.link = pyo.Constraint(expr=m.f == m.pw(m.x, m.y, m.w))

# 目标：最小化 f
m.obj = pyo.Objective(expr=m.f, sense=pyo.minimize)

# 把含有 PiecewiseLinearFunction 的模型转成 MIP（凸组合）
TransformationFactory('contrib.piecewise.convex_combination').apply_to(m)  # :contentReference[oaicite:3]{index=3}

# ----- 求解 -----
opt = pyo.SolverFactory('gurobi')   # 也可用 cbc/scip/cplex 等 MILP 求解器
res = opt.solve(m, tee=False)

print("status:", res.solver.termination_condition)
print("x, y, w:", pyo.value(m.x), pyo.value(m.y), pyo.value(m.w))
print("f* =", pyo.value(m.f))


status: optimal
x, y, w: 0.0 0.0 0.0
f* = 0.0


In [5]:
# 目标函数（我们用分段线性近似来表达它）
def f(y1,y2):
    return -np.sqrt(np.sqrt(y1**2 + y2**2))

# 生成 N 维三角剖分（在各维的 [lb, mid, ub] 上采样）
var_list = [m1.y1, m2.y3]
tri = pwutil.generate_delaunay(var_list, num=3)   # 每维3个点：lb/mid/ub
values = f(*tri.points.T)                         # 各格点上的函数值

# 多维分段线性关系（凸组合/cc 表示，通用 MILP，任何 MILP 求解器都可）
m.pw = piecewise_nd(tri, values, input=var_list, output=m.z, bound='eq', repn='cc')

# 一条简单约束，防止最优点退到(0,0,0)
#m.c1 = pmo.constraint(expr=m.x + m.y + m.w >= 1.2)

# 目标
m.obj = pmo.objective(expr=m.z, sense=pmo.minimize)

# ---------- 求解 ----------
res = pyo.SolverFactory("gurobi").solve(m, tee=False)   # 也可用 glpk/cbc/scip/cplex（MILP）
print("term:", res.solver.termination_condition)
print("x y w z* =", m.x.value, m.y.value, m.w.value, m.z.value)

NameError: name 'm1' is not defined

In [10]:
import itertools as it
import pyomo.environ as pyo
from pyomo.contrib.piecewise import PiecewiseLinearFunction
from pyomo.core.base import TransformationFactory

def add_nd_piecewise_pyo(model, x_vars, grids,
                         func=None, table=None,
                         name="pw", relation="==",
                         make_z_var=True, do_transform=True):
    assert (func is None) ^ (table is None)
    assert len(x_vars) == len(grids)
    # 节点集合
    point_list = list(it.product(*grids))

    if func is not None:
        pw = PiecewiseLinearFunction(points=point_list,
                                     function=lambda *cs: float(func(*cs)),
                                     name=f"{name}_fun")
    else:
        def _f_from_table(*cs):
            return float(table[tuple(float(c) for c in cs)])
        pw = PiecewiseLinearFunction(points=point_list,
                                     function=_f_from_table,
                                     name=f"{name}_fun")

    # 关键：先挂到模型
    model.add_component(pw.name, pw)

    # 形成表达式
    pw_expr = pw(*x_vars)

    if make_z_var:
        z = pyo.Var(name=f"{name}_z")
        model.add_component(z.name, z)
        if relation == "==":
            link = pyo.Constraint(expr=z == pw_expr)
        elif relation == ">=":
            link = pyo.Constraint(expr=z >= pw_expr)
        elif relation == "<=":
            link = pyo.Constraint(expr=z <= pw_expr)
        else:
            raise ValueError("relation must be '==', '>=', '<='")
        model.add_component(f"{name}_link", link)
        if do_transform:
            TransformationFactory('contrib.piecewise.convex_combination').apply_to(model)
        return z, pw
    else:
        expr = pyo.Expression(expr=pw_expr)
        model.add_component(f"{name}_expr", expr)
        if do_transform:
            TransformationFactory('contrib.piecewise.convex_combination').apply_to(model)
        return expr, pw


In [12]:
import numpy as np
import pyomo.environ as pyo

m = pyo.ConcreteModel()
m.y1 = pyo.Var(bounds=(-0.2, 0.2))
m.y2 = pyo.Var(bounds=(-0.2, 0.2))

grid = list(np.linspace(-0.2, 0.2, 3))  # [-0.2, 0.0, 0.2]
f = lambda a, b: np.sqrt(np.sqrt(a*a + b*b))

# 引入上面的通用函数
z, _ = add_nd_piecewise_pyo(
    m, [m.y1, m.y2], [grid, grid],
    func=f, name="pw2d", relation="==", make_z_var=True
)

m.obj = pyo.Objective(expr=z, sense=pyo.minimize)

res = pyo.SolverFactory("gurobi").solve(m, tee=False)
print(res.solver.termination_condition)
print("y1,y2,z* =", pyo.value(m.y1), pyo.value(m.y2), pyo.value(z))



optimal
y1,y2,z* = 0.0 0.0 0.0


In [ ]:
import itertools as it
import pyomo.environ as pyo
from pyomo.contrib.piecewise import PiecewiseLinearFunction
from pyomo.core.base import TransformationFactory

def add_nd_piecewise_pyo(model, x_vars, grids,
                         func=None, table=None,
                         name="pw", relation="==",
                         make_z_var=True, do_transform=True):
    assert (func is None) ^ (table is None)
    assert len(x_vars) == len(grids)
    # 节点集合
    point_list = list(it.product(*grids))

    if func is not None:
        pw = PiecewiseLinearFunction(points=point_list,
                                     function=lambda *cs: float(func(*cs)),
                                     name=f"{name}_fun")
    else:
        def _f_from_table(*cs):
            return float(table[tuple(float(c) for c in cs)])
        pw = PiecewiseLinearFunction(points=point_list,
                                     function=_f_from_table,
                                     name=f"{name}_fun")

    # 关键：先挂到模型
    model.add_component(pw.name, pw)

    # 形成表达式
    pw_expr = pw(*x_vars)

    if make_z_var:
        z = pyo.Var(name=f"{name}_z")
        model.add_component(z.name, z)
        if relation == "==":
            link = pyo.Constraint(expr=z == pw_expr)
        elif relation == ">=":
            link = pyo.Constraint(expr=z >= pw_expr)
        elif relation == "<=":
            link = pyo.Constraint(expr=z <= pw_expr)
        else:
            raise ValueError("relation must be '==', '>=', '<='")
        model.add_component(f"{name}_link", link)
        if do_transform:
            TransformationFactory('contrib.piecewise.convex_combination').apply_to(model)
        return z, pw
    else:
        expr = pyo.Expression(expr=pw_expr)
        model.add_component(f"{name}_expr", expr)
        if do_transform:
            TransformationFactory('contrib.piecewise.convex_combination').apply_to(model)
        return expr, pw

In [ ]:
import numpy as np
import pyomo.environ as pyo

m = pyo.ConcreteModel()
m.y1 = pyo.Var(bounds=(-0.2, 0.2))
m.y2 = pyo.Var(bounds=(-0.2, 0.2))

grid = list(np.linspace(-0.2, 0.2, 3))  # [-0.2, 0.0, 0.2]
f = lambda a, b: np.sqrt(np.sqrt(a*a + b*b))

# 引入上面的通用函数
z, _ = add_nd_piecewise_pyo(
    m, [m.y1, m.y2], [grid, grid],
    func=f, name="pw2d", relation="==", make_z_var=True
)

m.obj = pyo.Objective(expr=z, sense=pyo.minimize)

res = pyo.SolverFactory("gurobi").solve(m, tee=False)
print(res.solver.termination_condition)
print("y1,y2,z* =", pyo.value(m.y1), pyo.value(m.y2), pyo.value(z))

In [13]:
grid

[np.float64(-0.2), np.float64(0.0), np.float64(0.2)]

In [20]:
def add_nd_piecewise_from_points(
    model,
    firt_stg_vars,                 # [x1, x2, ..., xN]  模型里的 first stage Var
    points,                 # [(c11,...,c1N), (c21,...,c2N), ...]  所有节点（同维度 N）
    values,                 # 与 points 对齐的一维 list/array，或 dict{point_tuple: value}
    name="pw",
    relation="==",          # '==', '>='(下界/内逼近), '<='(上界/外逼近)
    round_ndigits=12,       # 为避免浮点比较问题，对坐标做轻微 round
):
    """
    返回 (z, pw)。z 是 Var（或你可将 make_z_var=False 改成返回 Expression）。
    """

    # 维度检查
    if len(points) == 0:
        raise ValueError("points 不能为空")
    N = len(firt_stg_vars)
    for pt in points:
        if len(pt) != N:
            raise ValueError(f"points 中出现与 x_vars 维度不一致的点: {pt}")

    # 统一坐标的浮点表示，避免查表时精度问题
    def keyize(coords):
        return tuple(round(float(c), round_ndigits) for c in coords)

    norm_points = [keyize(pt) for pt in points]

    # 把 values 统一成 dict 表
    if isinstance(values, dict):
        table = {keyize(k): float(v) for k, v in values.items()}
        # 确保每个 point 都有值
        miss = [pt for pt in norm_points if pt not in table]
        if miss:
            raise KeyError(f"values 缺少这些点的取值: {miss[:5]}{' ...' if len(miss)>5 else ''}")
    else:
        # 视作与 points 对齐的一维序列
        if len(values) != len(points):
            raise ValueError("values 长度应与 points 数量一致（或传 dict）")
        table = {pt: float(v) for pt, v in zip(norm_points, values)}

    # 查表函数（仅在节点上被调用）
    def _f_from_table(*coords):
        return table[keyize(coords)]

    # 创建并挂到模型
    pw = PiecewiseLinearFunction(points=norm_points, function=_f_from_table, name=f"{name}_fun")
    model.add_component(pw.name, pw)

    # 组成表达式
    pw_expr = pw(*firt_stg_vars)

    As = pyo.Var(name=f"{name}_As")
    model.add_component(As.name, As)
    if relation == "==":
        link = pyo.Constraint(expr=As == pw_expr)
    elif relation == ">=":
        link = pyo.Constraint(expr=As >= pw_expr)
    elif relation == "<=":
        link = pyo.Constraint(expr=As <= pw_expr)
    else:
        raise ValueError("relation 只能是 '==', '>=', '<='")
    model.add_component(f"{name}_link", link)

    TransformationFactory('contrib.piecewise.convex_combination').apply_to(model)
        
    return As, pw



In [23]:
import numpy as np
import pyomo.environ as pyo

# ---------- 模型 ----------
m = pyo.ConcreteModel()
m.y1 = pyo.Var(bounds=(-0.2, 0.2))
m.y2 = pyo.Var(bounds=(-0.2, 0.2))

grid = list(np.linspace(-0.2, 0.2, 5))  # [-0.2, 0.0, 0.2]
points = [(a,b) for a in grid for b in grid]  # 笛卡尔积 9 个点
values = [float(np.sqrt(np.sqrt(a*a + b*b))) for (a,b) in points]

# 用 add_nd_piecewise_from_points (points+values 版)
As, _ = add_nd_piecewise_from_points(
    m, [m.y1, m.y2], points, values,
    name="pw2d", relation="=="
)

m.obj = pyo.Objective(expr=As, sense=pyo.minimize)

res = pyo.SolverFactory("gurobi").solve(m, tee=False)
print(res.solver.termination_condition)
print("y1,y2,z* =", pyo.value(m.y1), pyo.value(m.y2), pyo.value(z))


optimal
y1,y2,z* = 0.0 0.0 0.0


In [24]:
points

[(np.float64(-0.2), np.float64(-0.2)),
 (np.float64(-0.2), np.float64(-0.1)),
 (np.float64(-0.2), np.float64(0.0)),
 (np.float64(-0.2), np.float64(0.10000000000000003)),
 (np.float64(-0.2), np.float64(0.2)),
 (np.float64(-0.1), np.float64(-0.2)),
 (np.float64(-0.1), np.float64(-0.1)),
 (np.float64(-0.1), np.float64(0.0)),
 (np.float64(-0.1), np.float64(0.10000000000000003)),
 (np.float64(-0.1), np.float64(0.2)),
 (np.float64(0.0), np.float64(-0.2)),
 (np.float64(0.0), np.float64(-0.1)),
 (np.float64(0.0), np.float64(0.0)),
 (np.float64(0.0), np.float64(0.10000000000000003)),
 (np.float64(0.0), np.float64(0.2)),
 (np.float64(0.10000000000000003), np.float64(-0.2)),
 (np.float64(0.10000000000000003), np.float64(-0.1)),
 (np.float64(0.10000000000000003), np.float64(0.0)),
 (np.float64(0.10000000000000003), np.float64(0.10000000000000003)),
 (np.float64(0.10000000000000003), np.float64(0.2)),
 (np.float64(0.2), np.float64(-0.2)),
 (np.float64(0.2), np.float64(-0.1)),
 (np.float64(0.2), np.